## Testing the ngram model

**We support:**
- n_grams of different length
- word vectors are weighted with tf-idf

**We need:**
- To be sure of the columns classes we want (Atm: `FIRTNAME, NAME, ADDRESS, CITY, DATE, CODE, ID, STRING`)
- A way to reduce the number of feature (maybe total number of occurence per feature)
- Scale to a column-like vision to enhance the distinction between STRING/FIRSTNAME/NAME/CODE to use redundancy: use number of different items
- Add more features: number of words, ending pattern (with voyelle), ratio voyelle/consonne
- In this vision, use the column name to confirm or enhance classification.


In [1]:
import loader
import ngram
from ipywidgets import FloatProgress

In [2]:
source = [
    ('firstname', 'firstnames.firstname', 100),
    
    ('name', 'names.name', 100),
    
    ('code', 'patients.gender', 10),
    ('code', 'admissions.marital_status', 10),
    ('code', 'admissions.religion', 10),
    ('code', 'admissions.insurance', 10),
    ('code', 'admissions.admission_location', 10),
    ('code', 'prescriptions.drug_type', 30),
    ('code', 'prescriptions.dose_unit_rx', 20),
    
    ('date', 'prescriptions.startdate', 90),
    ('date', 'admissions.admittime', 10),
    
    ('id', 'admissions.hadm_id', 10),
    ('id', 'admissions.subject_id', 10),
    ('id', 'prescriptions.subject_id', 80),
    
    ('address', 'addresses.road', 100),
    
    ('city', 'addresses.city', 100)
]

In [3]:
dataset = []
labels = []
for column in source:
    if len(column) >= 3:
        label, column_name, nb_datasets = column
    else:
        label, column_name, nb_datasets = column, 1
    dataset.append((column_name, nb_datasets))
    labels += [label.upper()] * nb_datasets

In [4]:
%%time

max_value = len(labels)
bar = FloatProgress(min=0, max=max_value)
display(bar)

columns = loader.fetch_columns(dataset, limit=100, load_bar=bar)

FloatProgress(value=0.0, max=700.0)

CPU times: user 1.53 s, sys: 218 ms, total: 1.75 s
Wall time: 4.73 s


In [7]:
clf = ngram.NGramClassifier()
X_train, y_train, X_test, y_test = clf.preprocess(columns, labels)

In [8]:
%%time

clf.fit(X_train, y_train, ngram_range=(2, 3))
y_pred = clf.predict(X_test)
clf.score(y_pred, y_test)

FIRSTNAME           	22/22	   100.0% 	(FP:0)
NAME                	24/24	   100.0% 	(FP:0)
CODE                	15/15	   100.0% 	(FP:0)
DATE                	11/11	   100.0% 	(FP:0)
ID                  	17/17	   100.0% 	(FP:0)
ADDRESS             	20/20	   100.0% 	(FP:0)
CITY                	31/31	   100.0% 	(FP:0)
SCORE 140/140 :   100.0%
CPU times: user 9.15 s, sys: 3.87 s, total: 13 s
Wall time: 13.9 s
